<img src="https://docs.actable.ai/_images/logo.png" style="object-fit: cover; max-width:100%; height:300px;" />

# infer_causal

This notebook is an example on how you can run a causal inference analyisis automatically with
[Actable AI](https://actable.ai)

For this example we will see the average effect of a treatment on an outcome\
and then see how controlling by a certain feature gives us multiple average effects.

### Imports

This part simply imports the python modules.
Last line imports the infer_causal from actableai

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from actableai import infer_causal
from actableai.utils.testing import unittest_hyperparameters

### Importing the data

This part simply imports the data.

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/Actable-AI/public-datasets/master/apartments.csv"
).head(100)
df

### Calling Actable AI task

This part is the call to the ActableAI causal inference analysis.\
To learn more about the available parameters you can consult the [API Documentation](https://lib.actable.ai/actableai.tasks.html#actableai.tasks.causal_inference.infer_causal)

In [ ]:
# Here df is the DataFrame containing our data
# target is "Churn" because we want to predict the churn
# features set to None means that we will use every single feature available
result = infer_causal(pd_table=df, treatments=["location"], outcomes=["rental_price"])


### Evaluation of the generated model

In this part we take a look at the metrics created by the model on the validation set.\
The validation set is created internally so you dont need to specify it.

In [ ]:
evaluation = result["data"]["evaluate"]
metrics = evaluation["metrics"]
print(metrics)
labels = evaluation["labels"]
confusion_matrix = result["data"]["evaluate"]["confusion_matrix"]
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=labels)
disp.plot()
plt.show()
auc_curve = result["data"]["evaluate"]["auc_curve"]
fpr, tpr, roc_auc = auc_curve["False Positive Rate"], auc_curve["True Positive Rate"], metrics[metrics["metric"] == "roc_auc"]["value"].iloc[0]
disp = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
disp.plot()
plt.show()

### Prediction with the generated model

Finally, we showcase how we can use the generated model to make further predictions\
on unseen data. Here in our case we already have the values but this works for any\
new incoming data points.

In [ ]:
model = result["model"]
prediction = model.predict(df_prediction)
df_prediction["Predicted Churn"] = prediction
df_prediction[["customerID", "gender", "TotalCharges", "Churn", "Predicted Churn"]].head(5)